# 01. PyTorch Workflow Exercise Template

The following is a template for the PyTorch workflow exercises.

It's only starter code and it's your job to fill in the blanks.

Because of the flexibility of PyTorch, there may be more than one way to answer the question.

Don't worry about trying to be *right* just try writing code that suffices the question.

In [1]:
# Import necessary libraries
import torch
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else 'cup'
device

'cuda'

## 1. Create a straight line dataset using the linear regression formula (`weight * X + bias`).
  * Set `weight=0.3` and `bias=0.9` there should be at least 100 datapoints total. 
  * Split the data into 80% training, 20% testing.
  * Plot the training and testing data so it becomes visual.

Your output of the below cell should look something like:
```
Number of X samples: 100
Number of y samples: 100
First 10 X & y samples:
X: tensor([0.0000, 0.0100, 0.0200, 0.0300, 0.0400, 0.0500, 0.0600, 0.0700, 0.0800,
        0.0900])
y: tensor([0.9000, 0.9030, 0.9060, 0.9090, 0.9120, 0.9150, 0.9180, 0.9210, 0.9240,
        0.9270])
```

Of course the numbers in `X` and `y` may be different but ideally they're created using the linear regression formula.

In [30]:
# Create the data parameters
weight = 0.3
bias = 0.9
start = 0
end = 3
step = 0.03

# Make X and y using linear regression feature
X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias

print(f"Number of X samples: {len(X)}")
print(f"Number of y samples: {len(y)}")
print(f"First 10 X & y samples:\nX: {X[:10]}\ny: {y[:10]}")

Number of X samples: 100
Number of y samples: 100
First 10 X & y samples:
X: tensor([[0.0000],
        [0.0300],
        [0.0600],
        [0.0900],
        [0.1200],
        [0.1500],
        [0.1800],
        [0.2100],
        [0.2400],
        [0.2700]])
y: tensor([[0.9000],
        [0.9090],
        [0.9180],
        [0.9270],
        [0.9360],
        [0.9450],
        [0.9540],
        [0.9630],
        [0.9720],
        [0.9810]])


In [31]:
# Split the data into training and testing
split = int(0.8 * len(X))
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]
len(X_train), len(y_train), len(X_test), len(y_test)

(80, 80, 20, 20)

In [32]:
# Plot the training and testing data 
def plot_predictions(train_data=X_train,
                     train_labels=y_train,
                     test_data=X_test,
                     test_labels=y_test,
                     predictions=None):
  plt.figure(figsize=(10, 7))
  plt.scatter(train_data, train_labels, c="b", s=4, label="Training Data")
  plt.scatter(test_data, test_labels, c="g", s=4, label="Testing Data")
  
  if predictions is not None:
    plt.scatter(test_data, predictions, c='r', s=4, label="Predictions")
  
  plt.legend(prop={"size": 14})
  
plot_predictions(X_train, y_train, X_test, y_test)

## 2. Build a PyTorch model by subclassing `nn.Module`. 
  * Inside should be a randomly initialized `nn.Parameter()` with `requires_grad=True`, one for `weights` and one for `bias`. 
  * Implement the `forward()` method to compute the linear regression function you used to create the dataset in 1. 
  * Once you've constructed the model, make an instance of it and check its `state_dict()`.
  * **Note:** If you'd like to use `nn.Linear()` instead of `nn.Parameter()` you can.

In [33]:
# Create PyTorch linear regression model by subclassing nn.Module
class LinearRegressionModel(torch.nn.Module):
  def __init__(self):
    super().__init__()

    self.linear_layer = torch.nn.Linear(in_features=1,
                                        out_features=1)
    
  def forward(self, X: torch.Tensor) -> torch.Tensor:
    return self.linear_layer(X)

torch.manual_seed(42)
model = LinearRegressionModel()
model, model.state_dict()

(LinearRegressionModel(
   (linear_layer): Linear(in_features=1, out_features=1, bias=True)
 ),
 OrderedDict([('linear_layer.weight', tensor([[0.7645]])),
              ('linear_layer.bias', tensor([0.8300]))]))

In [34]:
# Instantiate the model and put it to the target device
model.to(device)
next(model.parameters()).device

device(type='cuda', index=0)

## 3. Create a loss function and optimizer using `nn.L1Loss()` and `torch.optim.SGD(params, lr)` respectively. 
  * Set the learning rate of the optimizer to be 0.01 and the parameters to optimize should be the model parameters from the model you created in 2.
  * Write a training loop to perform the appropriate training steps for 300 epochs.
  * The training loop should test the model on the test dataset every 20 epochs.

In [35]:
# Create the loss function and optimizer
loss_fn = torch.nn.L1Loss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.01)

In [36]:
# Training loop
torch.manual_seed(42)

# Train model for 300 epochs
epochs = 300

# Send data to target device
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)

for epoch in range(epochs):
  ### Training

  # Put model in train mode
  model.train()

  # 1. Forward pass
  y_pred = model(X_train)

  # 2. Calculate loss
  loss = loss_fn(y_pred, y_train)

  # 3. Zero gradients
  optimizer.zero_grad()

  # 4. Backpropagation
  loss.backward()

  # 5. Step the optimizer
  optimizer.step()

  ### Perform testing every 20 epochs
  if epoch % 20 == 0:

    # Put model in evaluation mode and setup inference context 
    model.eval()
      # 1. Forward pass
    with torch.inference_mode():
      test_pred = model(X_test)
      # 2. Calculate test loss
      test_loss = loss_fn(test_pred, y_test)
      # Print out what's happening
      print(f"Epoch: {epoch} | Train loss: {loss:.3f} | Test loss: {test_loss:.3f}")

Epoch: 0 | Train loss: 0.486 | Test loss: 1.137
Epoch: 20 | Train loss: 0.162 | Test loss: 0.434
Epoch: 40 | Train loss: 0.084 | Test loss: 0.151
Epoch: 60 | Train loss: 0.058 | Test loss: 0.101
Epoch: 80 | Train loss: 0.033 | Test loss: 0.056
Epoch: 100 | Train loss: 0.008 | Test loss: 0.012
Epoch: 120 | Train loss: 0.007 | Test loss: 0.037
Epoch: 140 | Train loss: 0.007 | Test loss: 0.037
Epoch: 160 | Train loss: 0.007 | Test loss: 0.037
Epoch: 180 | Train loss: 0.007 | Test loss: 0.037
Epoch: 200 | Train loss: 0.007 | Test loss: 0.037
Epoch: 220 | Train loss: 0.007 | Test loss: 0.037
Epoch: 240 | Train loss: 0.007 | Test loss: 0.037
Epoch: 260 | Train loss: 0.007 | Test loss: 0.037
Epoch: 280 | Train loss: 0.007 | Test loss: 0.037


## 4. Make predictions with the trained model on the test data.
  * Visualize these predictions against the original training and testing data (**note:** you may need to make sure the predictions are *not* on the GPU if you want to use non-CUDA-enabled libraries such as matplotlib to plot).

In [37]:
# Make predictions with the model
model.eval()

with torch.inference_mode():
  y_preds = model(X_test)

y_preds

tensor([[1.6145],
        [1.6236],
        [1.6326],
        [1.6416],
        [1.6507],
        [1.6597],
        [1.6688],
        [1.6778],
        [1.6869],
        [1.6959],
        [1.7049],
        [1.7140],
        [1.7230],
        [1.7321],
        [1.7411],
        [1.7501],
        [1.7592],
        [1.7682],
        [1.7773],
        [1.7863]], device='cuda:0')

In [38]:
# Plot the predictions (these may need to be on a specific device)
plot_predictions(predictions=y_preds.cpu())

## 5. Save your trained model's `state_dict()` to file.
  * Create a new instance of your model class you made in 2. and load in the `state_dict()` you just saved to it.
  * Perform predictions on your test data with the loaded model and confirm they match the original model predictions from 4.

In [42]:
from pathlib import Path

# 1. Create models directory 
model_path = Path('models')
model_path.mkdir(parents=True, exist_ok=True)

# 2. Create model save path 
model_name = "pytorch-workflow_exercises.pth"
model_save_path = model_path / model_name
# 3. Save the model state dict
torch.save(obj=model.state_dict(), f=model_save_path)

In [44]:
# Create new instance of model and load saved state dict (make sure to put it on the target device)
loaded_model = LinearRegressionModel()

loaded_model.load_state_dict(torch.load(model_save_path))
loaded_model.to(device), loaded_model.state_dict()

(LinearRegressionModel(
   (linear_layer): Linear(in_features=1, out_features=1, bias=True)
 ),
 OrderedDict([('linear_layer.weight', tensor([[0.3014]], device='cuda:0')),
              ('linear_layer.bias', tensor([0.8913], device='cuda:0'))]))

(LinearRegressionModel(
   (linear_layer): Linear(in_features=1, out_features=1, bias=True)
 ),
 OrderedDict([('linear_layer.weight', tensor([[0.3014]], device='cuda:0')),
              ('linear_layer.bias', tensor([0.8913], device='cuda:0'))]))

In [45]:
# Make predictions with loaded model and compare them to the previous
loaded_model.eval()
with torch.inference_mode():
  loaded_model_preds = loaded_model(X_test)

y_preds == loaded_model_preds

tensor([[True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True],
        [True]], device='cuda:0')

In [46]:
plot_predictions(predictions=loaded_model_preds.cpu())